In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px

from textblob import TextBlob

pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
df = pd.read_csv("/kaggle/input/GP Orders - 5.csv",encoding="utf-8", delimiter=',')
nRow, nCol = df.shape
print(f'There are {nRow} rows and {nCol} columns')
df.head(5)

Rename column names for better EDA and Let's take a quick look at what the data looks like:

In [ ]:
df = df.rename(columns={'Order Number': 'Order_Number',"Order Status":"Order_Status", "Book Name":"Book_Name","Order Date & Time":"Order_Date","City":"Billing_City", "Payment Method":"Payment_Method", "Total items":"Total_Items", "Total weight (grams)": "Total_Weight"})
df.head(5)

In [ ]:
df.isnull().sum()

In [ ]:
df[df['Book_Name'].isna()]

In [ ]:
df[df['Billing_City'].isna()]

In [ ]:
df.dropna(inplace= True)

In [ ]:
nRow, nCol = df.shape
print(f'There are {nRow} rows and {nCol} columns')

In [ ]:
df.Order_Status.unique()

In [ ]:
df.Order_Status.value_counts()


In [ ]:
# Set the width and height of the figure
plt.figure(figsize=(14,5))
histogram= df.Order_Status.hist()

# Rotate the x-labels by 30 degrees, and keep the text aligned horizontally
plt.xticks(rotation=30, horizontalalignment="center")
plt.title("NO. of Orders vs Order Status")
plt.xlabel("Order Status")
plt.ylabel("Number of Orders")
plt.show()

Distribution graphs (histogram/bar graph) of sampled columns:

In [ ]:
df.Billing_City.nunique()

In [ ]:
df.Billing_City.value_counts()

In [ ]:
cities = df.Billing_City.unique()
cities

In [ ]:
#Convert Billing_City into lower case letters
df['Billing_City'] = df['Billing_City'].str.lower()
df['Billing_City'] = df['Billing_City'].str.strip()

In [ ]:
#Convert Book_Name into lower case letters
df['Book_Name'] = df['Book_Name'].str.lower()
df['Book_Name'] = df['Book_Name'].str.strip()
df

In [ ]:
df['Payment_Method'] = df['Payment_Method'].replace({"Cash on Delivery (COD)": "Cash on delivery"})
df.Payment_Method.value_counts().to_frame('counts')

In [ ]:
# Set the width and height of the figure
plt.figure(figsize=(14,5))

df.Payment_Method.value_counts().plot(kind='bar')
# Rotate the x-labels by 30 degrees, and keep the text aligned horizontally
plt.xticks(rotation=30, horizontalalignment="center")
plt.title("Payment Methods vs. Order Status")
plt.xlabel("Payment Method")
plt.ylabel("Number of Deliveries")

plt.show()

In [ ]:
#Shout out muhammadismail99 for this function

from itertools import chain
# return list from series of comma-separated strings
def chainer(s):
    return list(chain.from_iterable(s.str.split('/')))

# calculate lengths of splits
lens = df['Book_Name'].str.split('/').map(len)

# create new dataframe, repeating or chaining as appropriate
df = pd.DataFrame({'Order_Number': np.repeat(df['Order_Number'], lens),
                    'Order_Status': np.repeat(df['Order_Status'], lens),
                    'Book_Name': chainer(df['Book_Name']),
                    'Order_Date': np.repeat(df['Order_Date'], lens),
                    'Billing_City': np.repeat(df['Billing_City'], lens)})

In [ ]:
df = df.assign(Order_Books_Name=df.Book_Name.str.split("/")).explode("Book_Name")

In [ ]:
# Set the width and height of the figure
plt.figure(figsize=(14,5))

top_10_seller = df.Order_Books_Name.explode().value_counts()[:10].plot.bar()

df.Payment_Method.value_counts().plot(kind='bar')

# Rotate the x-labels by 30 degrees, and keep the text aligned horizontally
plt.xticks(rotation=30, horizontalalignment="center")
plt.title("Top 10 Best Selling Books")
plt.xlabel("Books")
plt.ylabel("Number of Orders")
plt.show()

Best Selling Book

In [ ]:
top_selling_books = df[df['Order_Status']=='Completed']['Book_Name'].value_counts()
top_selling_books.head(1)

In [ ]:

top10 = df["Book_Name"].value_counts().nlargest(10).to_frame()

graph = px.bar(top10, y =top10['Book_Name'], x = top10.index, title = 'Top 10 Books ')

graph.update_xaxes(title="Books")
graph.update_yaxes(title="Count")
graph.update_traces()
graph.show()

In [ ]:
df["Order_Date"] = pd.DatetimeIndex(df["Order_Date"])
df['date'] = df['Order_Date'].dt.date
df['time'] = df['Order_Date'].dt.time
df["Day_Name"] = df["Order_Date"].dt.day_name()
df["Week_Day"] = df["Order_Date"].dt.dayofweek
df["DayofYear"] = df["Order_Date"].dt.dayofyear
df["Month_Number"] = df["Order_Date"].dt.month
df["Month_Name"] = df["Order_Date"].dt.month_name()
df['year'] = df["Order_Date"].dt.year
df.sample(50)